In [72]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
import json
import os
load_dotenv()
import yfinance as yf
from langchain.tools import tool
import functools
from pydantic import BaseModel, JsonValue
from langchain_openai import ChatOpenAI
import faker

from typing import Any, Dict
from pydantic import BaseModel
from pydantic import RootModel
from langchain_core.output_parsers import PydanticOutputParser, JsonOutputParser
import pickle

In [73]:
load_dotenv()

True

In [74]:
DATA_REGISTRY = {}

In [75]:
@tool
def get_products() -> list[Dict[str, Any]]:
    """
    Get a list of products.

    Returns:
        list[Dict[str, Any]]: A list of product dictionaries.
    """
    fake = faker.Faker()
    products = []
    for _ in range(10):
        product = {
            "name": fake.word().title(),
            "price": round(fake.random_number(digits=2), 2),
            "description": fake.sentence(nb_words=10),
            "image_url": fake.image_url()
        }
        products.append(product)
    return products

@tool
def get_top_n_selling_products(n: int) -> list[Dict[str, Any]]:
    """
    Get the top N selling products.

    Args:
        n (int): The number of top selling products to retrieve.

    Returns:
        list[Dict[str, Any]]: A list of top N selling product dictionaries.
    """
    fake = faker.Faker()
    products = []
    for _ in range(n):
        product = {
            "name": fake.word().title(),
            "price": round(fake.random_number(digits=2), 2),
            "description": fake.sentence(nb_words=10),
            "image_url": fake.image_url(),
            "units_sold": fake.random_number(digits=3)
        }
        products.append(product)
    # Sort products by units_sold in descending order
    products.sort(key=lambda x: x["units_sold"], reverse=True)
    return products

# data = yf.Ticker("AAPL").history(period="1mo")

# @functools.lru_cache(maxsize=None) # Unlimited cache size
def _get_stock_history(ticker: str, period: str = "5d"):
    data = yf.Ticker(ticker).history(period=period)
    return data


@tool
def get_stock_price(ticker: str) -> str:
    """
    Get the latest stock price for a given ticker symbol.
    Args:
        ticker (str): The stock ticker symbol.
    Returns:
        str: The latest stock price.

    """
    data = _get_stock_history(ticker, "1d")
    if data.empty:
        return f"No data found for ticker symbol: {ticker}"
    latest_price = data['Close'].iloc[-1]
    return latest_price

@tool
def get_stock_history(ticker: str, period: str = "5d") -> Dict[str, Any]:
    """
    This tool MUST be used whenever the user asks for stock history.
    Never answer directly. Always call this tool.
    Args:
        ticker (str): The stock ticker symbol.
        period (str): The period over which to retrieve historical data (e.g., '1mo', '3mo', '1y').
    Returns:
        str: A JSON string containing stock_history and metadata.
    """

    print(f"Fetching history for {ticker} over period {period}")

    data = _get_stock_history(ticker, period)

    if data.empty:
        return f"No data found for ticker symbol: {ticker}"
    
    data.index = data.index.strftime('%Y-%m-%d')
    data = data.reset_index()
    # round numeric columns to 2 decimal places
    data["Close"] = data["Close"].round(2)

    data_key=f"{ticker}_{period}"
    DATA_REGISTRY[data_key] = data

    history = data[['Date','Close']].head(3).to_dict(orient='records')


    metadata = {
                "ticker": ticker, 
                "period": period,
                "data_points": len(history),
                "columns": ['Date','Close'],
                "data_key": data_key,
                "data_sample_for_reference": history
                }
    return metadata


tool_registry = {
    "get_products": get_products,
    "get_top_n_selling_products": get_top_n_selling_products,
    "get_stock_price": get_stock_price,
    "get_stock_history": get_stock_history
}

In [76]:
# get_stock_history("AAPL", "1mo")

In [77]:
llm_google = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)


llm = ChatOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://aibe.mygreatlearning.com/openai/v1",
    model='gpt-4o-mini',
    temperature=0
)

ui_generation_llm = ChatOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://aibe.mygreatlearning.com/openai/v1",
    model='gpt-4o',
    temperature=0,
           streaming=True
)
# .bind(
#     # May instruct the LLM, but DO NOT send json_schema
#     response_format={"type": "json_object"}  
# )


# ui_generation_llm = ChatGoogleGenerativeAI(
#     model="gemini-2.5-flash",
#     temperature=0,
#     max_tokens=None,
#     timeout=None,
#     max_retries=2,
# )


In [78]:
ui_generation_llm

ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x335b72720>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x335b72f30>, root_client=<openai.OpenAI object at 0x33594ac30>, root_async_client=<openai.AsyncOpenAI object at 0x335b73f50>, model_name='gpt-4o', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://aibe.mygreatlearning.com/openai/v1', streaming=True)

In [79]:
class IndividualTask(BaseModel):
    task_id: str
    description: str
    confidence_score: float

class TaskList(BaseModel):
    task : list[IndividualTask]


planning_agent_system_message = """You are a Task Extracting Agent responsible for extracting individual task from a combined user query. 
Only extract tasks present in the combined user query. Do not create new tasks that are not explicitly mentioned in the user goal.
If the combined user query is already simple and does not require further breakdown, respond with a single task that reflects the original query.
Your response should be a list of sub-tasks in JSON format as shown below:
{{
    {"task": [
        {"task_id": "1", "description": "First sub-task description", confidence_score: 0.9},
        {"task_id: "2", "description": "Second sub-task description", confidence_score: 0.8},
        ... 
    ]}
}}
"""

In [80]:
llm_planning_agent = llm.with_structured_output(TaskList)

messages = [
    {"role": "system", "content": planning_agent_system_message},
    {"role": "user", "content": "Show me list of all the product available in the store and provide me the details of the top 5 products based on customer ratings."}
]

# messages = [
#     {"role": "system", "content": planning_agent_system_message},
#     {"role": "user", "content": "Get me the stock price history for AAPL for last 1 month and also provide me the latest stock price."}
# ]

response = llm_planning_agent.invoke(messages)

In [81]:
print(response.model_dump_json(indent=2))

{
  "task": [
    {
      "task_id": "1",
      "description": "Show the list of all products available in the store",
      "confidence_score": 0.9
    },
    {
      "task_id": "2",
      "description": "Provide details of the top 5 products based on customer ratings",
      "confidence_score": 0.9
    }
  ]
}


In [82]:
tasks = response.model_dump().get("task", [])
for task in tasks:
    print(f"Task ID: {task['task_id']}, Description: {task['description']}, Confidence Score: {task['confidence_score']}")

Task ID: 1, Description: Show the list of all products available in the store, Confidence Score: 0.9
Task ID: 2, Description: Provide details of the top 5 products based on customer ratings, Confidence Score: 0.9


In [83]:
tool_calling_agent_system_message = """
You are a Tool Calling Agent. Your goal is to determine if any of the tasks provided can be accomplished using the available tools and call the appropriate tool with the necessary parameters.
You have access to the following tools:
1. get_products: This tool retrieves a list of products available in the store.
2. get_top_n_selling_products: This tool retrieves the top N selling products based on customer ratings.
"""

tools = [get_products,get_top_n_selling_products,get_stock_price,get_stock_history]

tool_calling_agent = llm.bind_tools(tools)

In [84]:
tool_results = []
for task in tasks:
    task_description = task['description']
    print(f"\nProcessing Task ID: {task['task_id']}, Description: {task_description}")
    response = tool_calling_agent.invoke([
        {"role": "system", "content": tool_calling_agent_system_message},
        {"role": "user", "content": task_description}
    ])
    print(f"Response for Task ID {task['task_id']}:\n{response}\n")
    if response.tool_calls:
        for tool_call in response.tool_calls:
            name = tool_call["name"]
            args = tool_call["args"]

            tool = tool_registry.get(name)
            if tool is None:
                print(f"⚠️ Unknown tool requested: {name}")
                continue

            # Invoke dynamically
            result = tool.invoke(args)

            print("Tool result:", result)
            tool_results.append({
                "task_description": task_description,
                "tool_name": name,
                "args": args,
                "tool_result": result
            })


Processing Task ID: 1, Description: Show the list of all products available in the store
Response for Task ID 1:
content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 353, 'total_tokens': 363, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_11f3029f6b', 'id': 'chatcmpl-ClDqhMkus1RPxQuFw6JSkqDbQ8Q96', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--0efe89cc-dfd1-4fd4-b787-28f806a94ef5-0' tool_calls=[{'name': 'get_products', 'args': {}, 'id': 'call_bO2NHtYMM8nPWbn2PoPOtoE6', 'type': 'tool_call'}] usage_metadata={'input_tokens': 353, 'output_tokens': 10, 'total_tokens': 363, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'out

In [85]:
tool_results

[{'task_description': 'Show the list of all products available in the store',
  'tool_name': 'get_products',
  'args': {},
  'tool_result': [{'name': 'Phone',
    'price': 8,
    'description': 'Alone art class size pull where improve drive change seem without leg against.',
    'image_url': 'https://dummyimage.com/340x520'},
   {'name': 'Consumer',
    'price': 52,
    'description': 'Rather various reach above type bit southern.',
    'image_url': 'https://placekitten.com/653/601'},
   {'name': 'Everything',
    'price': 5,
    'description': 'Drop product industry hundred system example couple attack.',
    'image_url': 'https://placekitten.com/965/461'},
   {'name': 'Stage',
    'price': 1,
    'description': 'Dinner investment seem probably call marriage gas first program stop act.',
    'image_url': 'https://placekitten.com/227/383'},
   {'name': 'Matter',
    'price': 24,
    'description': 'Build economic future prove they pass couple seem avoid final.',
    'image_url': 'https

In [86]:
print(json.dumps(tool_results, indent=2))

[
  {
    "task_description": "Show the list of all products available in the store",
    "tool_name": "get_products",
    "args": {},
    "tool_result": [
      {
        "name": "Phone",
        "price": 8,
        "description": "Alone art class size pull where improve drive change seem without leg against.",
        "image_url": "https://dummyimage.com/340x520"
      },
      {
        "name": "Consumer",
        "price": 52,
        "description": "Rather various reach above type bit southern.",
        "image_url": "https://placekitten.com/653/601"
      },
      {
        "name": "Everything",
        "price": 5,
        "description": "Drop product industry hundred system example couple attack.",
        "image_url": "https://placekitten.com/965/461"
      },
      {
        "name": "Stage",
        "price": 1,
        "description": "Dinner investment seem probably call marriage gas first program stop act.",
        "image_url": "https://placekitten.com/227/383"
      },
     

In [87]:
from typing import List, Literal, Optional, Union
from pydantic import BaseModel, Field

# --- Base and Auxiliary Models ---

class BaseComponent(BaseModel):
    # This must be defined as str first, and then overridden by Literals in subclasses
    type: str
    # usage: Optional[str] = None

    model_config = {
        "extra": "forbid"
    }

class TableCell(BaseModel):
    key: str
    # Note: Using `str | int | float | bool | None` is the modern Pydantic/Python way
    value: str | int | float | bool | None

class TableRow(BaseModel):
    cells: List[TableCell]

# --- Component Definitions (Uses string literal "Component" for recursion) ---

class Container(BaseComponent):
    type: Literal["container"]
    variant: Literal["vertical", "horizontal"]
    children: List["Component"]


class Text(BaseComponent):
    type: Literal["text"]
    variant: Literal["header", "subheader", "paragraph"]
    value: str


class Chart(BaseComponent):
    type: Literal["chart"]
    variant: Literal["line", "bar"]
    x_axis: str
    y_axis: str
    title: str
    dataKey: str

class Table(BaseComponent):
    type: Literal["table"]
    title: str
    # columns: List[str]
    # rows: List[TableRow]
    dataKey: str


class Card(BaseComponent):
    type: Literal["card"]
    title: str
    content: str


class Button(BaseComponent):
    type: Literal["button"]
    label: str
    action: str


class Image(BaseComponent):
    type: Literal["image"]
    src: str
    alt: str


class ListComponent(BaseComponent):
    type: Literal["list"]
    children: List["Component"]

class Root(BaseComponent):
    type: Literal["root"]
    children: List["Component"]

# --- Component Union Definition ---

# Define the Union using the model classes
Component = Union[
    Root,
    Container,
    Text,
    Chart,
    Table,
    Card,
    Button,
    Image,
    ListComponent,
]

# --- Resolve Forward References ---

# The rebuild calls are ESSENTIAL to link the string "Component" 
# to the actual Component Union type.
Root.model_rebuild()
Container.model_rebuild()
# Card.model_rebuild()
ListComponent.model_rebuild()

# --- Final Output Schema ---

class UISpec(BaseModel):
    root: Root

    model_config = {
        "extra": "forbid"
    }

basic_ui_spec = """
{
    "type": "root",
    "usage": "Use 'root' as the top-level container for the entire UI.",
    "children": [ ...components ]
},
{
  "type": "container",
  "usage": "Use 'container' to group related components together.",
  "variant": "vertical" | "horizontal",
  "children": [ ...components ]
},
{
  "type": "text",
  "variant": "header" | "subheader" | "paragraph",
  "usage": "Use 'text' to display static text content.",
  "value": "string"
},
{
  "type": "chart",
  "usage": "Use 'chart' to visualize data in different formats. Choose the column names for x_axis and y_axis based on the data provided.",
  "variant": "line" | "bar",
  "x_axis": "string",
  "y_axis": "string",
  "title": "string",
  "dataKey": "chart_data_key"
},
{
  "type": "table",
  "usage": "Use 'table' to display tabular data. Depending on the data, you can have different columns and rows.",
  "title": "string",
  "dataKey": "table_stock_data"
},
{
  "type": "card",
  "usage": "Use 'card' to present information in a concise format.",
  "title": "string",
  "content": "string"
},
{
  "type": "button",
  "usage": "Use 'button' to trigger actions or events.",
  "label": "string",
  "action": "string"
},
{
  "type": "image",
  "usage": "Use 'image' to display visual content.",
  "src": "image_url",
  "alt": "string"
},
{
  "type": "list",
  "usage": "Use 'list' to display a collection of items.",
  "children": [ ...components ]
}
"""

In [88]:


intermediate_ui_spec = """
{
  "type": "input",
  "usage": "Use 'input' to capture user input.",
  "inputType": "text" | "number" | "date",
  "placeholder": "string",
  "dataKey": "input_data_key"
},
{
  "type": "form",
  "usage": "Use 'form' to collect multiple inputs from users.",
  "title": "string",
  "fields": [ ...input components ]
},
{
  "type": "accordion",
  "usage": "Use 'accordion' to organize content into expandable sections.",
  "sections": [
    {
      "title": "string",
      "content": [ ...components ]
    }
  ]
},
{
  "type": "carousel",
  "usage": "Use 'carousel' to display a series of images or content.",
  "items": [
    {
      "imageSrc": "image_url",
      "caption": "string"
    }
  ]
}
"""

advanced_ui_spec = """
{
  "type": "map",
  "usage": "Use 'map' to visualize geographical data.",
  "markers": [
    {
      "latitude": "number",
      "longitude": "number",
      "label": "string"
    }
  ]
}
"""


ui_generation_system_message = f"""
You are a highly specialized **UI Generation Agent** designed to create high-fidelity UI specifications. Your primary task is to translate user requirements and provided JSON data into a structured, executable UI definition.

**Core Instruction:** Generate a single, comprehensive JSON object that represents the complete UI specification.

### Component Library & Structure Rules

Use only the following components and determine when to use each based on the component 'usage' description provided below:
{basic_ui_spec}

### Output Principles

1.  **Structure:** The final JSON object **MUST** strictly adhere to the defined Pydantic schema (the `UISpec` model). The top-level key must be `root`. And output MUST be compacted, containing no extraneous whitespace, newlines, or indentation.
2.  **Data Integration:** When data is required for a `chart` or `table`, utilize the `dataKey` field to specify the required data set. Do **NOT** invent row/cell data in the output unless it is for static examples.
3.  **Hierarchy:** Components must be organized logically within nested `container` and `root` elements to enhance clarity and visual grouping.
4.  **Actionability:** Every `button` must have an explicit, clear `label` that indicates its purpose.
"""

ui_generation_system_message_v2 = """
You are a highly specialized UI Generation Agent.

Your task is to generate a complete UI specification that follows these principles:

1. Choose the appropriate UI components based on the meaning of the user data.
2. Organize components into logical containers to create a clear hierarchy.
3. Use charts and tables only when the data supports them.
4. For charts and tables, never invent data — always reference the provided dataKey.
5. Ensure all button labels clearly express intent.
6. The final reply should contain only the UI specification as required by the UISpec structured output schema.

Do NOT output example JSON, component definitions, or explanations.

"""


structured_ui_gen_llm = ui_generation_llm.with_structured_output(UISpec, method="function_calling")
# parser = JsonOutputParser()

messages = [
    {"role": "system", "content": ui_generation_system_message},
    {"role": "user", "content": f"Generate a UI specification for the following data:\n  {json.dumps(tool_results, ensure_ascii=False)}"}
]
# res = structured_ui_gen_llm.invoke(messages)



In [58]:
json.dumps(tool_results)

'[{"task_description": "Get the stock price history for AAPL for the last 1 month", "tool_name": "get_stock_history", "args": {"ticker": "AAPL", "period": "1mo"}, "tool_result": {"ticker": "AAPL", "period": "1mo", "data_points": 3, "columns": ["Date", "Close"], "data_key": "AAPL_1mo", "data_sample_for_reference": [{"Date": "2025-11-10", "Close": 269.43}, {"Date": "2025-11-11", "Close": 275.25}, {"Date": "2025-11-12", "Close": 273.47}]}}, {"task_description": "Provide the latest stock price for AAPL", "tool_name": "get_stock_price", "args": {"ticker": "AAPL"}, "tool_result": 277.17999267578125}]'

In [59]:
messages

[{'role': 'system',
  'content': '\nYou are a highly specialized UI Generation Agent.\n\nYour task is to generate a complete UI specification that follows these principles:\n\n1. Choose the appropriate UI components based on the meaning of the user data.\n2. Organize components into logical containers to create a clear hierarchy.\n3. Use charts and tables only when the data supports them.\n4. For charts and tables, never invent data — always reference the provided dataKey.\n5. Ensure all button labels clearly express intent.\n6. The final reply should contain only the UI specification as required by the UISpec structured output schema.\n\nDo NOT output example JSON, component definitions, or explanations.\n\n'},
 {'role': 'user',
  'content': 'Generate a UI specification for the following data:\n  [{"task_description": "Get the stock price history for AAPL for the last 1 month", "tool_name": "get_stock_history", "args": {"ticker": "AAPL", "period": "1mo"}, "tool_result": {"ticker": "AA

In [89]:
chunks = []
for chunk in structured_ui_gen_llm.stream(messages):
        chunks.append(chunk)
        print(chunk.model_dump(), end="", flush=True)

{'root': {'type': 'root', 'children': []}}{'root': {'type': 'root', 'children': [{'type': 'container', 'variant': 'vertical', 'children': []}]}}{'root': {'type': 'root', 'children': [{'type': 'container', 'variant': 'vertical', 'children': [{'type': 'text', 'variant': 'header', 'value': ''}]}]}}{'root': {'type': 'root', 'children': [{'type': 'container', 'variant': 'vertical', 'children': [{'type': 'text', 'variant': 'header', 'value': 'Product'}]}]}}{'root': {'type': 'root', 'children': [{'type': 'container', 'variant': 'vertical', 'children': [{'type': 'text', 'variant': 'header', 'value': 'Product Catalog'}]}]}}{'root': {'type': 'root', 'children': [{'type': 'container', 'variant': 'vertical', 'children': [{'type': 'text', 'variant': 'header', 'value': 'Product Catalog'}, {'type': 'container', 'variant': 'horizontal', 'children': []}]}]}}{'root': {'type': 'root', 'children': [{'type': 'container', 'variant': 'vertical', 'children': [{'type': 'text', 'variant': 'header', 'value': 'Pr

In [90]:
chunks

[UISpec(root=Root(type='root', children=[])),
 UISpec(root=Root(type='root', children=[Container(type='container', variant='vertical', children=[])])),
 UISpec(root=Root(type='root', children=[Container(type='container', variant='vertical', children=[Text(type='text', variant='header', value='')])])),
 UISpec(root=Root(type='root', children=[Container(type='container', variant='vertical', children=[Text(type='text', variant='header', value='Product')])])),
 UISpec(root=Root(type='root', children=[Container(type='container', variant='vertical', children=[Text(type='text', variant='header', value='Product Catalog')])])),
 UISpec(root=Root(type='root', children=[Container(type='container', variant='vertical', children=[Text(type='text', variant='header', value='Product Catalog'), Container(type='container', variant='horizontal', children=[])])])),
 UISpec(root=Root(type='root', children=[Container(type='container', variant='vertical', children=[Text(type='text', variant='header', value='P

In [91]:
print(chunks[-1].model_dump_json(indent=2))

{
  "root": {
    "type": "root",
    "children": [
      {
        "type": "container",
        "variant": "vertical",
        "children": [
          {
            "type": "text",
            "variant": "header",
            "value": "Product Catalog"
          },
          {
            "type": "container",
            "variant": "horizontal",
            "children": [
              {
                "type": "list",
                "children": [
                  {
                    "type": "container",
                    "variant": "vertical",
                    "children": [
                      {
                        "type": "image",
                        "src": "https://dummyimage.com/340x520",
                        "alt": "Phone"
                      },
                      {
                        "type": "text",
                        "variant": "subheader",
                        "value": "Phone"
                      },
                      {
             

In [92]:
chunks_json = [chunk.model_dump() for chunk in chunks]

In [93]:
with open("ui_spec_stream.json", "w") as f:
    json.dump(chunks_json, f, indent=2)